In [21]:
import neuroglancer
import cloudvolume
import json, csv
import graphviz

In [50]:
# neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')
neuroglancer.set_static_content_source(url='http://localhost:8080')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
print(viewer)

http://127.0.0.1:43592/v/dc60dc860b847d63277459745268b060305d975f/


In [3]:
# get dictionary representing atlas
ontology_file = '/home/ahoag/ngdemo/graphviz/allen_ontology.json'

# test_ontology_file = 'test_ontology.json'
# with open(test_ontology_file) as json_file:
#     test_data = json.load(json_file)
    
# ontology_file = 'allen_ontology.json'
with open(ontology_file) as json_file:
    data = json.load(json_file)

In [4]:
# First need a make_graph() function to make the entire graph for visualization purposes
def make_graph(dic,graph):
    """ Make a edge-unweighted directed graph from a dictionary
    Representing a brain ontology
    """
    name = dic.get('name')
    children = dic.get('children')
    for child in children:
        child_name = child.get('name')
        graph.edge(name,child_name)
        make_graph(child,graph)
    return 

In [5]:
G = graphviz.Digraph()
make_graph(data,G)
G

In [6]:
# some helper functions
def get_progeny(dic,input_nodename,progeny_list):
    """
    Gets all of the descendents of a given input nodename
    """
    name = dic.get('name')
    children = dic.get('children')
    if name == input_nodename:
        for child in children: # child is a dict
            child_name = child.get('name')
            progeny_list.append(child_name)
            get_progeny(child,input_nodename=child_name,progeny_list=progeny_list)
        return
    
    for child in children:
        child_name = child.get('name')
        get_progeny(child,input_nodename=input_nodename,progeny_list=progeny_list)
    return progeny_list

def get_parent(graph,input_nodename):
    if len(input_nodename.split(' ')) > 1:
        nodename_to_search = f'"{input_nodename}"'
    else:
        nodename_to_search = input_nodename
    edges_pointing_to_node=[x for x in G.body if f'-> {nodename_to_search}' in x]
    if len(edges_pointing_to_node) == 0:
        return None
    elif len(edges_pointing_to_node) > 1:
        print("Error. There should not be more than one edge pointing to this node")
    else:
        parent_nodename = edges_pointing_to_node[0].split('->')[0].strip()
        # remove the extra quotes surrounding the nodename if there is more than one word
        if len(parent_nodename.split(' ')) > 1:
            return parent_nodename[1:-1]
        else:
            return parent_nodename
    return

In [7]:
get_parent(G,"Cerebrum")

'Basic cell groups and regions'

In [8]:
get_parent(G,"Simple lobule, granular layer")

'Simple lobule'

In [9]:
# Now load in the dictionary mapping ID to region name
atlas_csv_file = '/home/ahoag/ngdemo/graphviz/allen_id_table_w_voxel_counts.csv'
with open(atlas_csv_file,'r') as infile:
    reader = csv.reader(infile)
    next(reader) # skip header
    ontology_id_dict = {int(rows[3]):rows[1] for rows in reader}
ontology_id_dict

{997: 'root',
 8: 'Basic cell groups and regions',
 567: 'Cerebrum',
 688: 'Cerebral cortex',
 695: 'Cortical plate',
 315: 'Isocortex',
 184: 'Frontal pole, cerebral cortex',
 68: 'Frontal pole, layer 1',
 667: 'Frontal pole, layer 2/3',
 526157192: 'Frontal pole, layer 5',
 526157196: 'Frontal pole, layer 6a',
 526322264: 'Frontal pole, layer 6b',
 500: 'Somatomotor areas',
 107: 'Somatomotor areas, Layer 1',
 219: 'Somatomotor areas, Layer 2/3',
 299: 'Somatomotor areas, Layer 5',
 644: 'Somatomotor areas, Layer 6a',
 947: 'Somatomotor areas, Layer 6b',
 985: 'Primary motor area',
 320: 'Primary motor area, Layer 1',
 943: 'Primary motor area, Layer 2/3',
 648: 'Primary motor area, Layer 5',
 844: 'Primary motor area, Layer 6a',
 882: 'Primary motor area, Layer 6b',
 993: 'Secondary motor area',
 656: 'Secondary motor area, layer 1',
 962: 'Secondary motor area, layer 2/3',
 767: 'Secondary motor area, layer 5',
 1021: 'Secondary motor area, layer 6a',
 1085: 'Secondary motor area, 

In [10]:
get_progeny(data,input_nodename='Medial group of the dorsal thalamus',progeny_list=[])

['Intermediodorsal nucleus of the thalamus',
 'Mediodorsal nucleus of thalamus',
 'Mediodorsal nucleus of the thalamus, central part',
 'Mediodorsal nucleus of the thalamus, lateral part',
 'Mediodorsal nucleus of the thalamus, medial part',
 'Submedial nucleus of the thalamus',
 'Perireunensis nucleus']

In [11]:
# key binding to merge segment for a region with its parent segment using the ontology

num_actions = 0
def my_action(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['atlas']
        region_name = ontology_id_dict[region_id]
        print(region_id,region_name)
        # Look up parent name and then get corresponding ID
        parent_name = get_parent(G,region_name)
        if parent_name:
            print("Found parent node")
            print(parent_name)
            for ID,name in ontology_id_dict.items():
                if name == parent_name:
                    parent_id = ID
                    print("Found parent ID:")
                    print(parent_id)
                    break
            else:
                print("Had trouble finding ID given parent name")
                return
        else:
            print("Could not find parent node")
            return
    # find all progeny
    progeny_list = get_progeny(data,input_nodename=parent_name,progeny_list=[])
    equivalence_list = [(region_id, parent_id), (parent_id, parent_id)] 
#     for progeny in progeny_list:
#         for ID,name in ontology_id_dict.items():
#             if name == progeny:
#                 progeny_id = ID
#                 equivalence_list.append((progeny_id,parent_id)) 
#                 break
#     print(equivalence_list)
    with viewer.txn() as s:
        existing_equivalences = list(s.layers['atlas'].layer.equivalences.items())
        final_equivalence_list = existing_equivalences + equivalence_list
        s.layers['atlas'].layer.equivalences = final_equivalence_list
#         region_name = atlas_fake_dict.get(region_id)
#         st.status_messages['hello'] = ('%i:%s' %
#                                      (region_id, region_name))
#     print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
#     print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action', my_action)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action'
#     s.status_messages['hello'] = 'Welcome to the parent merge first example. Press p to use.'


In [12]:
print(viewer)

http://127.0.0.1:43592/v/01b12fcf668c2336779fc132997aa629201f1675/


In [53]:
with viewer.txn() as s:
    print(s.layers['atlas'].layer.equivalences.items())

dict_items([(59, 59), (149, 59), (599, 59)])


In [52]:
with viewer.txn() as s:
    s.layers['atlas'].layer.equivalences = [(59, 599), (149, 599)]

In [54]:
k1 = 0xcc9e2d51;

In [55]:
int(k1)

3432918353

In [59]:
import operator

In [64]:
59 * k1

-50860887637

In [61]:
import numpy as np

In [63]:
np.int32(59)

59

In [65]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'